# Evaluation

Steps:
1. Harmonize the predictions to have the same format
2. Extract the frequencies using a sliding window approach
3. Evaluate the performance of the models
4. Visualize the results

## Step 1: Harmonize the predictions

In [ ]:
from respiration.analysis import PredictionsReader

predictions = PredictionsReader().read_all()
len(predictions)

In [ ]:
# Show all models
predictions['model'].unique()

## Step 2: Extract the frequencies using a sliding window approach

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
subject = 'Proband24'
setting = '101_natural_lighting'
setting2 = '303_normalized_face'

In [ ]:
sampling_rate = 30
lowpass = 0.1
highpass = 0.5

In [ ]:
from respiration.analysis import (
    butterworth_filter,
    normalize_signal,
)

models = [
    # 'lucas_kanade',
    # 'SimpleViT_20240729_195756',
    'RF_20240903_051739',
    'RF_20240904_001421',
    # 'raft_small',
    # 'pixel_intensity_grey',
    # 'tf_20240729_195756',
    # 'MMPD_intra_RhythmFormer',
    # 'mtts_can',
    # 'big_small',
]

signals = []

for model in models:
    print(f'{model}:')
    
    if model == 'RF_20240904_001421':
        set = setting2
    else:
        set = setting
    
    prediction_x = predictions[
        (predictions['subject'] == subject) &
        (predictions['setting'] == set) &
        (predictions['model'] == model)].iloc[0]['signal']

    print(f'{model}: {prediction_x.shape}')

    # Normalize the signals
    prediction = normalize_signal(prediction_x)

    # Filter the signals
    prediction = butterworth_filter(prediction, sampling_rate, lowpass, highpass)

    # Add the signals to the list
    signals.append({
        'label': model,
        'signal': prediction,
        'signal_x': prediction_x,
    })

In [ ]:
# Get the ground truth signal
gt_signal_xxx = dataset.get_breathing_signal(subject, setting)
gt_signal = dataset.get_breathing_signal(subject, setting)
gt_signal = normalize_signal(gt_signal)
gt_signal = butterworth_filter(gt_signal, sampling_rate, lowpass, highpass)

In [ ]:
# Plot the gt_signal_xxx signal
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 5))
plt.plot(gt_signal_xxx)
plt.title('Ground truth signal')
plt.show()

In [ ]:
# Create an interactive plot
import numpy as np
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(len(gt_signal)), y=gt_signal, mode='lines', name='Ground truth'))

for signal in signals:
    fig.add_trace(
        go.Scatter(x=np.arange(len(signal['signal'])), y=signal['signal'], mode='lines', name=signal['label']))

fig.show()

In [ ]:
from respiration.analysis import frequency_from_psd

gt_freq = frequency_from_psd(gt_signal, sampling_rate)
print(f'Ground truth signal {gt_freq:.2f} ({gt_freq * 60:.2f} bpm)')